In [1]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 28kB/s 
     |████████████████████████████████| 491kB 60.0MB/s 
     |████████████████████████████████| 3.2MB 53.5MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [2]:


from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
import torch

from keras.utils import np_utils
from keras.callbacks import History 
from keras.models import load_model

from datetime import datetime

import tensorflow as tf


import os
from google.colab import drive

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:

drive.mount('/content/drive')

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"
# COMP_PATH = 'C:/Users/moong/Desktop/CS5242 Project/'
# COMP_PATH = 'C:\\Users\\Calvin\\Documents\\Python Scripts\\NUS\\CS5242 Neural Networks and Deep Learning\\Project'

os.chdir(COMP_PATH)
os.listdir()

tf.__version__

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'1.14.0'

In [0]:
def add_padding(video_segment, target_len):

    video_len = len(video_segment)
    padded_video_segment = video_segment
    if video_len > 0:
        if video_len < target_len:
            padded_video_segment = np.zeros((target_len, 400))
            padded_video_segment[0:len(video_segment)] = video_segment
        elif (video_len > target_len):
            # Get the middle part
            start_idx = (video_len-target_len)//2
            padded_video_segment = video_segment[start_idx:start_idx+target_len]
        else:
            padded_video_segment = video_segment

    return padded_video_segment

def add_video_by_len(curr_data, label_curr_video, data_breakfast, labels_breakfast, target_len):
    # print(curr_data)
    # print(label_curr_video)

    idx = 0
    video_segment = []
    curr_label = label_curr_video[0]
    for i, label in enumerate(label_curr_video):
        if label != curr_label:
            if curr_label != 0:
                data_breakfast.append(add_padding(video_segment, target_len))
                labels_breakfast.append(curr_label)
            curr_label = label
            video_segment = []
            idx = 0
        video_segment.append(curr_data[i])
        idx += 1
        if idx == target_len:
            data_breakfast.append(add_padding(video_segment, target_len))
            labels_breakfast.append(curr_label)
            curr_label = label
            video_segment = []
            idx = 0

    if curr_label != 0:
        data_breakfast.append(add_padding(video_segment, target_len))
        labels_breakfast.append(curr_label)

    return data_breakfast, labels_breakfast
    
def add_video_segment(video_segment, segment_label, all_video_segments, all_video_labels, target_len):
    video_len = len(video_segment)
    if (video_len > 0) and (segment_label != 0):
        if (video_len < target_len):
            padded_video_segment = np.zeros((target_len, 400))
            padded_video_segment[0:len(video_segment)] = video_segment
        elif (video_len > target_len):
            # Get the middle part
            start_idx = (video_len-target_len)//2
            padded_video_segment = video_segment[start_idx:start_idx+target_len]
        else:
            padded_video_segment = video_segment
        all_video_segments.append(padded_video_segment)
        all_video_labels.append(segment_label)
    return all_video_segments, all_video_labels

def get_video_segment_by_len(video_group_list, label_group_list, target_len):
    all_video_segments = []
    all_video_labels = []
    for i, label_group in enumerate(label_group_list):
        video_group = video_group_list[i]
        video_segment = []
        segment_label = label_group_list[0]
        
        idx = 0
        for j, label in enumerate(label_group):
            
            if (label != segment_label):
                all_video_segments, all_video_labels = add_video_segment(
                    video_segment, segment_label, all_video_segments, all_video_labels, target_len)
                segment_label = label
                video_segment = []
                idx = 0
            video_segment.append(video_group[j])
            idx += 1
            if idx == target_len:
                all_video_segments, all_video_labels = add_video_segment(
                    video_segment, segment_label, all_video_segments, all_video_labels, target_len)
                video_segment = []
                idx = 0

        all_video_segments, all_video_labels = add_video_segment(
            video_segment, segment_label, all_video_segments, all_video_labels, target_len)
    
    return np.array(all_video_segments), np.array(all_video_labels)
    
def load_data(split_load, actions_dict, GT_folder, DATA_folder, datatype = 'training', target_len = 100, start_file_to_load = 0, last_file_to_load = 10):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
    
    if datatype == 'training':
        data_breakfast = []
        labels_breakfast = []
        
        # print(len(content_all))
        for i, content in enumerate(content_all):
        # for content in content_all:
            if i < start_file_to_load:
                continue
            elif i <= last_file_to_load:
                # file_ptr = open( GT_folder + content, 'r')
                # curr_gt = file_ptr.read().split('\n')[:-1]
                # label_seq, length_seq = get_label_length_seq(curr_gt)

                loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
                curr_data = np.loadtxt(loc_curr_data, dtype='float32')
                label_curr_video = []
                for iik in range(len(curr_gt)):
                    label_curr_video.append( actions_dict[curr_gt[iik]] )

                data_breakfast, labels_breakfast = add_video_by_len(curr_data, label_curr_video, data_breakfast, labels_breakfast, target_len)
            else:
                break
            
        print("Finish Load the Training data and labels!!!")
        return np.array(data_breakfast), np.array(labels_breakfast)

        # return  data_breakfast, labels_uniq

        # return  data_breakfast, labels_breakfast
    if datatype == 'test':
        data_breakfast = []
        for content in content_all:
        
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            
            data_breakfast.append(curr_data)

        print("Finish Load the Test data!!!")
        return data_breakfast
        
def load_data_v1(split_load, actions_dict, GT_folder, DATA_folder, datatype = 'training', target_len = 100):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
    
    if datatype == 'training':
        data_breakfast = []
        labels_breakfast = []
        for content in content_all:
        
            file_ptr = open( GT_folder + content, 'r')
            curr_gt = file_ptr.read().split('\n')[:-1]
            # label_seq, length_seq = get_label_length_seq(curr_gt)

            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            label_curr_video = []
            for iik in range(len(curr_gt)):
                label_curr_video.append( actions_dict[curr_gt[iik]] )
         
            data_breakfast.append(curr_data)
            labels_breakfast.append(label_curr_video)
    
        # labels_uniq, labels_uniq_loc = get_label_bounds(labels_breakfast)

        print("getting test segments")
        all_video_segments, all_video_labels = get_video_segment_by_len(data_breakfast, labels_breakfast, target_len)

        print("Finish Load the Training data and labels!!!")     
        return all_video_segments, all_video_labels

        # return  data_breakfast, labels_uniq

        # return  data_breakfast, labels_breakfast
    if datatype == 'test':
        data_breakfast = []
        for content in content_all:
        
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            
            data_breakfast.append(curr_data)


        print("Finish Load the Test data!!!")
        return data_breakfast


def find_end_idx(y_train, start_idx, target_len):
    orig_start_idx = start_idx
    curr_label = y_train[start_idx]
    # print(curr_label)
    counter = 0
    # print("start_idx:", start_idx)
    # print("adfdasfdd:", target_len)
    while counter < target_len and \
        start_idx < len(y_train) and \
        curr_label == y_train[start_idx]:
        # print(start_idx)
        # print(y_train[start_idx])
        start_idx += 1
        counter += 1

    if counter >= target_len:
        return orig_start_idx + target_len
    elif start_idx >= len(y_train):
        return len(y_train)
    else:
        return start_idx

def pad_x(x, target_len):
    padded_x = np.zeros((target_len, 1, 400))
    padded_x[0:len(x)] = x
    return padded_x

def pad_y(y, target_len):
    padded_y = np.zeros((target_len, 1))
    padded_y[0:len(y)] = y
    return padded_y

def one_hot_to_label(one_hot_list):
    return np.argmax(one_hot_list, axis=-1)

def load_segment_file(segment_file):
    file_ptr = open(segment_file, 'r')
    file_content = file_ptr.read().split('\n')[0:-1]
    test_segment_tmp = [file_content[i].split(' ') for i in range(len(file_content))]
    test_segment_all = []
    for i, test_segment in enumerate(test_segment_tmp):
        segment_list = []
        for j, segment_idx in enumerate(test_segment):
            segment_list.append(int(segment_idx))
        test_segment_all.append(segment_list)
    return test_segment_all

def find_seg_end_idx(segment_list, start_idx, target_len):
    curr_idx = 0
    for i, curr_segment_idx in enumerate(segment_list):
        if curr_segment_idx <= start_idx:
            continue
        else:
            break
    segment_len = curr_segment_idx - start_idx
    if segment_len > target_len:
        return start_idx + target_len
    else:
        return curr_segment_idx

def find_majority_label(label_list, start_idx, end_idx):
    label_count_dict = {}
    for i in range(start_idx, end_idx, 1):
        label = label_list[i]
        if label in label_count_dict:
            label_count_dict[label] += 1
        else:
            label_count_dict[label] = 1
    max_count = 0
    max_label = -1
    for i, label in enumerate(label_count_dict):
        print("===== Label {}; count is: {}".format(label, label_count_dict[label]))
        if label_count_dict[label] > max_count:
            max_count = label_count_dict[label]
            max_label = label
    return max_label

def find_max_count_label_in_dict(label_count_dict):
    max_count = 0
    max_label = 0
    for label, count in label_count_dict.items():
        if count > max_count:
            max_count = count
            max_label = label
    return max_label

def find_max_segment_len(video_seg_loc):
    max_len = 0
    for i in range(len(video_seg_loc)-1):
        this_len = video_seg_loc[i+1] - video_seg_loc[i]
        if this_len > max_len:
            max_len = this_len
    return max_len

    

In [5]:
from read_datasetBreakfast import read_mapping_dict, load_data

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"

''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
# train_split =  os.path.join(COMP_PATH, 'splits/2 train.split1.bundle') #Train Split
# train_split =  os.path.join(COMP_PATH, 'splits/valid.split1.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/2 valid.split1.bundle') #Train Split 
# train_split =  os.path.join(COMP_PATH, 'splits/300 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/90 valid.split1 - P49_54.bundle') # Valid Split 
# train_split =  os.path.join(COMP_PATH, 'splits/900 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/150 valid.split1 - P49_54.bundle') # Valid Split 
train_split =  os.path.join(COMP_PATH, 'splits/train.split1 - Orig.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
# test_split  =  os.path.join(COMP_PATH, 'splits/10 test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 
test_segment_file =  os.path.join(COMP_PATH, 'segment.txt') # segment file

target_len = 3000
start_file_to_load = 0
last_file_to_load = 1460

actions_dict = read_mapping_dict(mapping_loc)
# for split in ['training','test']:
for split in ['training']:
    if  split == 'training':
        startTime = datetime.now()
        # X_data, Y_data = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split, target_len = target_len, start_file_to_load=start_file_to_load, last_file_to_load=last_file_to_load) #Get features and labels
        X_data, Y_data, video_seg_loc_list = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
        # data_feat_train, data_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
        # labels_uniq, labels_uniq_loc = get_label_bounds(data_labels)
        endTime = datetime.now()
        print("******** Total Time to load file:",(endTime - startTime))
    if  split == 'test':
        X_test = load_data( test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only
        test_segment_list = load_segment_file(test_segment_file)



# Y_data = np.reshape(Y_data, (Y_data.shape[0], 1))
# X_test = np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1]))

# print(len(data_feat))
# print(len(data_labels))
# print(data_feat)
# print(data_labels)

Finish Load the Training data and labels!!!
******** Total Time to load file: 0:34:46.336740


In [0]:

# for i, video_sequence in enumerate(X_data):
#     label_sequence = Y_data[i]
#     video_seg_loc = video_seg_loc_list[i]

#     print(label_sequence)
#     print(video_seg_loc)

#     max_len = find_max_segment_len(video_seg_loc)
#     curr_seg = len(video_seg_loc)-1
#     start_idx = min(video_seg_loc)
#     end_idx = max(video_seg_loc)
#     next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)

#     print(num_of_seg)
#     print(start_idx)
#     print(end_idx)
#     print(next_idx)
#     print(len(video_sequence))

#     curr_video = 0
#     while True:

#         x_train = np.array([video_sequence[start_idx]])
#         y_train = np.array([Y_data[curr_seg]])

#         x_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1]))
#         y_train = np.reshape(y_train, (1, y_train.shape[0], y_train.shape[1]))

#         print(x_train.shape)
#         print(y_train.shape)

#         start_idx += 1

#         if start_idx >= len(video_sequence) or start_idx == end_idx:
#             break
#         elif start_idx == next_idx:
#             next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)
#             print("next_idx: {}".format(next_idx))
#             curr_seg += 1


#     break


[1, 2]
[9, 269, 474]
2
9
474
269
544
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 10)
(1, 1, 400)
(1, 1, 1

In [0]:
target_len = 3000

# n_hidden_nodes = target_len//2
n_hidden_nodes = 200
n_dense_nodes = 50
n_features = 400
n_timesteps = target_len
n_outputs = 48

n_epoch = 1
batch_size = 1
init_lr = 0.01

clipnorm=1.0
clipvalue=0.5

input_dropout = 0.4
recurrent_dropout = 0.4

num_of_itr = total_len//500
# num_of_itr = 1000

#Step 1 Inital Build Model and Start Training

In [0]:


from keras import optimizers
from keras import regularizers

# from keras.models import Model, Sequential
# from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, BatchNormalization, TimeDistributed, Flatten



lr = tf.compat.v1.train.exponential_decay(
    init_lr,                # Base learning rate.
    n_epoch,  # Current index into the dataset.
    100000,          # Decay step.
    0.9,                # Decay rate.
    staircase=True)

adam = optimizers.Adam(lr=0.005, clipnorm=clipnorm, clipvalue=clipvalue)



model = Sequential()
# model.add(Bidirectional(LSTM(n_hidden_nodes, 
#                              input_shape=(n_timesteps, n_features), 
#                              kernel_initializer='random_uniform',
#                              recurrent_initializer='random_uniform',
#                             #  kernel_regularizer=regularizers.l2(0.01), 
#                             #  recurrent_regularizer=regularizers.l2(0.01) 
#                              )))
# model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features))))
model.add(LSTM(n_hidden_nodes, batch_input_shape=(batch_size, None, n_features), stateful=True))
# model.add(LSTM(n_hidden_nodes, return_sequences=True, batch_input_shape=(batch_size, None, n_features), stateful=True))
# model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features), dropout=input_dropout, recurrent_dropout=recurrent_dropout)))
model.add(BatchNormalization())

# model.add(LSTM(100))

model.add(Dense(n_dense_nodes_1, activation='relu'))

model.add(Dense(n_dense_nodes_2, activation='relu'))

model.add(Dense(n_outputs, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer=adam)
# model.compile(loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'], optimizer=adam)


history = History()

# start_idx = 0
# end_idx = start_idx + target_len
total_loss = 0
total_acc = 0
counter = 0

overall_startTime = datetime.now()
for j in range(n_epoch):
    total_loss = 0
    total_acc = 0
    counter = 0
    epoch_itr = 0

    epoch_startTime = datetime.now()
    for i, video_sequence in enumerate(X_train):
        label_sequence = Y_train[i]
        # print(label_sequence)
        video_seg_loc = video_seg_loc_list_train[i]
        # print(video_seg_loc)

        video_startTime = datetime.now()

        itr_startTime = datetime.now()

        # print(label_sequence)
        # print(video_seg_loc)

        max_len = find_max_segment_len(video_seg_loc)

        curr_seg = 0
        start_idx = min(video_seg_loc)
        if len(video_seg_loc) == 1:
            end_idx = len(video_sequence)
            next_idx = len(video_sequence)
        else:
            end_idx = max(video_seg_loc)
            next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)

        # print(num_of_seg)
        # print(start_idx)
        # print(end_idx)
        # print(next_idx)
        # print(len(video_sequence))

        while True and len(label_sequence) > 0:

            x = np.array([video_sequence[start_idx]])
            y = np.array([label_sequence[curr_seg]])


            y = np_utils.to_categorical(y, num_classes=n_outputs)





            x = np.reshape(x, (1, x.shape[0], x.shape[1]))
            # print(y_train.shape)
            # y = np.reshape(y, (1, y.shape[0], 1))            
            y = np.reshape(y, (y.shape[0], n_outputs))            

            history = model.fit(x, y, epochs=n_epoch, batch_size=batch_size, verbose=0)

            # print(history.history.keys())
            # break
            train_loss_list = history.history['loss']
            # train_acc_list  = history.history['sparse_categorical_accuracy']
            train_acc_list  = history.history['categorical_accuracy']

            for k, train_loss in enumerate(train_loss_list):
                total_loss += train_loss
                total_acc += train_acc_list[k]
                counter += 1
                average_loss = total_loss / counter
                average_acc = total_acc / counter

            if (epoch_itr % num_of_itr == 0):
                local_loss = total_loss/counter
                local_acc = total_acc/counter
                # print('***** Iteration {}, ===== acc {:.2f}, loss {:.2f}\n'.format(epoch_itr, local_acc, local_loss))

                itr_endTime = datetime.now()

                for l, train_loss in enumerate(train_loss_list):
                    # print('***** Label {}, ===== start_idx {}, end_idx {}'.format(curr_label, start_idx, end_idx))
                    print('***** Iteration {}, Iteration time took: {}; ===== acc {:.3f}, loss {:.3f}'.format(epoch_itr, str(itr_endTime-itr_startTime), train_acc_list[l], train_loss))
                    print('***** Overall time took: {}; Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(str(itr_endTime-overall_startTime), average_acc, average_loss))

                itr_startTime = datetime.now()

            epoch_itr += 1

            start_idx += 1
            if start_idx >= len(video_sequence) or start_idx == end_idx:
                model.reset_states()
                break
            elif start_idx == next_idx:
                model.reset_states()
                next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)
                # print("next_idx: {}".format(next_idx))
                curr_seg += 1

            

        video_endTime = datetime.now()

        print('***** Video {} Finished, This video time took: {}\n'.format(i, str(video_endTime-video_startTime)))

    
    epoch_endTime = datetime.now()

    average_train_loss = total_loss/counter
    average_train_acc = total_acc/counter

    print('***** Final Running Average Overall Time took: {}; ===== acc {:.3f}, loss {:.3f}\n'.format(str(epoch_endTime-overall_startTime), average_train_acc, average_train_loss))
    
    

print('\n# Evaluate on validation data')
for i, video_sequence in enumerate(X_valid):
    label_sequence = Y_valid[i]
    # print(label_sequence)
    video_seg_loc = video_seg_loc_list_valid[i]

    # print(label_sequence)
    # print(video_seg_loc)

    max_len = find_max_segment_len(video_seg_loc)

    curr_seg = 0
    start_idx = min(video_seg_loc)
    if len(video_seg_loc) == 1:
        end_idx = len(video_sequence)
        next_idx = len(video_sequence)
    else:
        end_idx = max(video_seg_loc)
        next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)

    # print(num_of_seg)
    # print(start_idx)
    # print(end_idx)
    # print(next_idx)
    # print(len(video_sequence))
    label_count_dict = {}
    while True and len(label_sequence) > 0:

        x = np.array([video_sequence[start_idx]])
        y = np.array([label_sequence[curr_seg]])
        true_label = label_sequence[curr_seg]

        y = np_utils.to_categorical(y, num_classes=n_outputs)




        x = np.reshape(x, (1, x.shape[0], x.shape[1]))
        # print(y_train.shape)
        # y = np.reshape(y, (1, y.shape[0], 1))            
        y = np.reshape(y, (y.shape[0], n_outputs))            

        history = model.evaluate(x, y, batch_size=batch_size, verbose=0)

        y_predict = model.predict_classes(x, batch_size=batch_size, verbose=2)
        pred_label = int(y_predict[0])

        total_loss += history[0]
        total_acc += history[1]
        counter += 1

        if pred_label in label_count_dict:
            label_count_dict[pred_label] +=1
        else:
            label_count_dict[pred_label] = 1

        start_idx += 1
        if start_idx >= len(video_sequence) or start_idx == end_idx:
            majority_label = find_max_count_label_in_dict(label_count_dict)
            print("***** y_true: {}; y_pred: {}".format(true_label, str(majority_label)))

            break
        elif start_idx == next_idx:
            majority_label = find_max_count_label_in_dict(label_count_dict)
            print("***** y_true: {}; y_pred: {}".format(true_label, str(majority_label)))
            next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)
            # print("next_idx: {}".format(next_idx))
            curr_seg += 1

    # model.reset_states()

average_valid_loss = total_loss/counter
average_valid_acc = total_acc/counter

print('***** Final Evaluation Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(average_valid_acc, average_valid_loss))
print('Validation loss, Validation acc:', history)


model.summary()

model.save('20200405_v2_LSTM_Main_trial_2.h5')








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
***** Iteration 0, Iteration time took: 0:00:01.869376; ===== acc 0.000, loss 3.871
***** Overall time took: 0:00:01.869836; Running Average ===== acc 0.000, loss 3.871

***** Iteration 300, Iteration time took: 0:00:02.652810; ===== acc 0.000, loss 3.418
***** Overall time took: 0:00:04.523254; Running Average ===== acc 0.860, loss 2.808

***** Video 0 Finished, This video time took: 0:00:05.938798

***** Iteration 600, Iteration time took: 0:00:01.172371; ===== acc 0.000, loss 2.272
***** Overall time took: 0:00:07.111757; Running Average ===== acc 0.431, loss 2.791

***** Iteration 900, Iteration time took: 0:00:02.459936; ===== acc 0.000, loss 2.984
***** Overall time took: 0:00:09.571874; Running Average ===== acc 0.434, loss 2.633

***** Iteration 1200, Iteration time took: 0:00:02.515921; ===== acc 0.000, loss 2.425
***** Overall time took: 0:00:12.087954; Running Average ===== ac

# Step 2 Load model and continue Training



In [0]:


from keras import optimizers
from keras import regularizers

# from keras.models import Model, Sequential
# from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, BatchNormalization, TimeDistributed, Flatten


# model_to_load = os.path.join(COMP_PATH, "Saved Models/20200405_v2_LSTM_Main_trial_2.h5")

# model = load_model(model_to_load)

# start_idx = 0
# end_idx = start_idx + target_len
total_loss = 0
total_acc = 0
counter = 0

overall_startTime = datetime.now()
for j in range(n_epoch):
    total_loss = 0
    total_acc = 0
    counter = 0
    epoch_itr = 0

    epoch_startTime = datetime.now()
    for i, video_sequence in enumerate(X_train):
        label_sequence = Y_train[i]
        # print(label_sequence)
        video_seg_loc = video_seg_loc_list_train[i]
        # print(video_seg_loc)

        video_startTime = datetime.now()

        itr_startTime = datetime.now()

        # print(label_sequence)
        # print(video_seg_loc)

        max_len = find_max_segment_len(video_seg_loc)

        curr_seg = 0
        start_idx = min(video_seg_loc)
        if len(video_seg_loc) == 1:
            end_idx = len(video_sequence)
            next_idx = len(video_sequence)
        else:
            end_idx = max(video_seg_loc)
            next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)

        # print(num_of_seg)
        # print(start_idx)
        # print(end_idx)
        # print(next_idx)
        # print(len(video_sequence))

        while True and len(label_sequence) > 0:

            x = np.array([video_sequence[start_idx]])
            y = np.array([label_sequence[curr_seg]])


            y = np_utils.to_categorical(y, num_classes=n_outputs)


            x = np.reshape(x, (1, x.shape[0], x.shape[1]))
            # print(y_train.shape)
            # y = np.reshape(y, (1, y.shape[0], 1))            
            y = np.reshape(y, (y.shape[0], n_outputs))            

            history = model.fit(x, y, epochs=n_epoch, batch_size=batch_size, verbose=0)

            # print(history.history.keys())
            # break
            train_loss_list = history.history['loss']
            # train_acc_list  = history.history['sparse_categorical_accuracy']
            train_acc_list  = history.history['categorical_accuracy']

            for k, train_loss in enumerate(train_loss_list):
                total_loss += train_loss
                total_acc += train_acc_list[k]
                counter += 1
                average_loss = total_loss / counter
                average_acc = total_acc / counter

            if (epoch_itr % num_of_itr == 0):
                local_loss = total_loss/counter
                local_acc = total_acc/counter
                # print('***** Iteration {}, ===== acc {:.2f}, loss {:.2f}\n'.format(epoch_itr, local_acc, local_loss))

                itr_endTime = datetime.now()

                for l, train_loss in enumerate(train_loss_list):
                    # print('***** Label {}, ===== start_idx {}, end_idx {}'.format(curr_label, start_idx, end_idx))
                    print('***** Iteration {}, Iteration time took: {}; ===== acc {:.3f}, loss {:.3f}'.format(epoch_itr, str(itr_endTime-itr_startTime), train_acc_list[l], train_loss))
                    print('***** Overall time took: {}; Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(str(itr_endTime-overall_startTime), average_acc, average_loss))

                itr_startTime = datetime.now()

            epoch_itr += 1

            start_idx += 1
            if start_idx >= len(video_sequence) or start_idx == end_idx:
                model.reset_states()
                break
            elif start_idx == next_idx:
                model.reset_states()
                next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)
                # print("next_idx: {}".format(next_idx))
                curr_seg += 1

        video_endTime = datetime.now()

        print('***** Video {} Finished, This video time took: {}\n'.format(i, str(video_endTime-video_startTime)))

    
    epoch_endTime = datetime.now()

    average_train_loss = total_loss/counter
    average_train_acc = total_acc/counter

    print('***** Final Running Average Overall Time took: {}; ===== acc {:.3f}, loss {:.3f}\n'.format(str(epoch_endTime-overall_startTime), average_train_acc, average_train_loss))


model.summary()

model.save('20200405_v2_LSTM_Main_trial_2.h5')


***** Iteration 0, Iteration time took: 0:00:00.011533; ===== acc 0.000, loss 6.914
***** Overall time took: 0:00:00.011899; Running Average ===== acc 0.000, loss 6.914

***** Video 0 Finished, This video time took: 0:00:01.935017

***** Iteration 500, Iteration time took: 0:00:02.025133; ===== acc 1.000, loss 1.526
***** Overall time took: 0:00:03.960643; Running Average ===== acc 0.062, loss 3.093

***** Iteration 1000, Iteration time took: 0:00:04.033053; ===== acc 0.000, loss 16.118
***** Overall time took: 0:00:07.993887; Running Average ===== acc 0.263, loss 6.015

***** Iteration 1500, Iteration time took: 0:00:04.012522; ===== acc 1.000, loss 1.014
***** Overall time took: 0:00:12.006594; Running Average ===== acc 0.189, loss 7.454

***** Iteration 2000, Iteration time took: 0:00:03.956167; ===== acc 1.000, loss 0.089
***** Overall time took: 0:00:15.962940; Running Average ===== acc 0.391, loss 5.661

***** Iteration 2500, Iteration time took: 0:00:04.004782; ===== acc 0.000, 

In [0]:
model_to_load = os.path.join(COMP_PATH, "Saved Models/20200405_v2_LSTM_Main_trial_2.h5")

model = load_model(model_to_load)


print('\n# Evaluate on validation data')
for i, video_sequence in enumerate(X_valid):
    label_sequence = Y_valid[i]
    # print(label_sequence)
    video_seg_loc = video_seg_loc_list_valid[i]

    # print(label_sequence)
    # print(video_seg_loc)

    max_len = find_max_segment_len(video_seg_loc)

    curr_seg = 0
    start_idx = min(video_seg_loc)
    if len(video_seg_loc) == 1:
        end_idx = len(video_sequence)
        next_idx = len(video_sequence)
    else:
        end_idx = max(video_seg_loc)
        next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)

    # print(num_of_seg)
    # print(start_idx)
    # print(end_idx)
    # print(next_idx)
    # print(len(video_sequence))
    label_count_dict = {}
    while True and len(label_sequence) > 0:

        x = np.array([video_sequence[start_idx]])
        y = np.array([label_sequence[curr_seg]])
        true_label = label_sequence[curr_seg]

        x = np.reshape(x, (1, x.shape[0], x.shape[1]))
        # print(y_train.shape)
        # y = np.reshape(y, (1, y.shape[0], 1))            
        y = np.reshape(y, (y.shape[0], 1))            

        # history = model.evaluate(x, y, batch_size=batch_size, verbose=0)
        y_predict = model.predict_classes(x, batch_size=batch_size, verbose=2)
        pred_label = y_predict[0]

        if pred_label in label_count_dict:
            label_count_dict[pred_label] +=1
        else:
            label_count_dict[pred_label] = 1

        start_idx += 1
        if start_idx >= len(video_sequence) or start_idx == end_idx:
            majority_label = find_max_count_label_in_dict(label_count_dict)
            print("***** y_true: {}; y_pred: {}; {}".format(true_label, str(majority_label), y_predict))
            break
        elif start_idx == next_idx:
            majority_label = find_max_count_label_in_dict(label_count_dict)
            print("***** y_true: {}; y_pred: {}; {}".format(true_label, str(majority_label), y_predict))
            next_idx = find_seg_end_idx(video_seg_loc, start_idx, max_len)
            # print("next_idx: {}".format(next_idx))
            curr_seg += 1

    # model.reset_states()

# average_valid_loss = total_loss/counter
# average_valid_acc = total_acc/counter

# print('***** Final Evaluation Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(average_valid_acc, average_valid_loss))
# print('Validation loss, Validation acc:', history)




# Evaluate on validation data
***** y_true: 10; y_pred: 29; [27]
***** y_true: 11; y_pred: 27; [27]
***** y_true: 12; y_pred: 27; [27]
***** y_true: 13; y_pred: 27; [27]
***** y_true: 14; y_pred: 43; [27]
***** y_true: 15; y_pred: 27; [27]
***** y_true: 18; y_pred: 30; [30]
***** y_true: 19; y_pred: 27; [27]
***** y_true: 20; y_pred: 27; [30]
***** y_true: 21; y_pred: 27; [5]
***** y_true: 20; y_pred: 27; [5]
***** y_true: 21; y_pred: 27; [36]
***** y_true: 20; y_pred: 27; [27]
***** y_true: 21; y_pred: 27; [27]
***** y_true: 20; y_pred: 27; [27]
***** y_true: 21; y_pred: 27; [33]
***** y_true: 24; y_pred: 27; [27]
***** y_true: 2; y_pred: 27; [5]
***** y_true: 25; y_pred: 27; [27]
***** y_true: 2; y_pred: 27; [44]
***** y_true: 11; y_pred: 27; [27]
***** y_true: 26; y_pred: 27; [36]
***** y_true: 2; y_pred: 27; [33]
***** y_true: 27; y_pred: 27; [18]
***** y_true: 26; y_pred: 27; [27]
***** y_true: 27; y_pred: 27; [33]
***** y_true: 28; y_pred: 27; [27]
***** y_true: 29; y_pred: 30; 

In [0]:
target_len = 3000

# n_hidden_nodes = target_len//2
n_hidden_nodes = 200
n_dense_nodes_1 = 100
n_dense_nodes_2 = 50
n_features = 400
n_timesteps = target_len
n_outputs = 48

n_epoch = 1
batch_size = 1
init_lr = 0.01

clipnorm=1.0
clipvalue=0.5

input_dropout = 0.4
recurrent_dropout = 0.4

# num_of_itr = total_len//500
num_of_itr = 500

In [7]:
ratio = 0.8
split_idx = int(len(X_data)*ratio)
X_train_data = X_data[0:split_idx]
X_valid = X_data[split_idx:len(X_data)]
video_seg_loc_list_train_data = video_seg_loc_list[0:split_idx]
video_seg_loc_list_valid = video_seg_loc_list[split_idx:len(X_data)]
Y_train_data = Y_data[0:split_idx]
Y_valid = Y_data[split_idx:len(X_data)]

print(len(X_train_data))
print(len(X_valid))
print(len(video_seg_loc_list_train_data))
print(len(video_seg_loc_list_valid))
print(len(Y_train_data))
print(len(Y_valid))

# X_train_data = X_train
# Y_train_data = Y_train
# video_seg_loc_list_train_data = video_seg_loc_list_train

total_len = 0
for i in range(len(X_train_data)):
    total_len += len(X_train_data[i])

print("total_len: {}". format(total_len))



# for i in range(len(Y_data)):
#     print(len(Y_data[i]))
#     print(Y_data[i])

# print(Y_data)

# 0.85 = 1241:219
# total_len = 2,661,234

1168
292
1168
292
1168
292
total_len: 2490253


In [21]:

split = 5
curr_round = 2
batch_len = len(X_train_data)//split
curr_start = batch_len * (curr_round - 1)

if curr_round != split:
    X_train = X_train_data[curr_start:batch_len*curr_round]
    Y_train = Y_train_data[curr_start:batch_len*curr_round]
    video_seg_loc_list_train = video_seg_loc_list_train_data[curr_start:batch_len*curr_round]
else:
    X_train = X_train_data[curr_start:len(X_train_data)]
    Y_train = Y_train_data[curr_start:len(X_train_data)]
    video_seg_loc_list_train = video_seg_loc_list_train_data[curr_start:len(X_train_data)]

batch_total_len = 0
for i in range(len(X_train)):
    batch_total_len += len(X_train[i])

print("batch_total_len: {}".format(batch_total_len))


batch_total_len: 508696


In [0]:
# a = []
# while True:
#     a.append("1")